In [27]:
# k-nearest neighbor

# Import packages
import pandas as pd
import numpy as np
from collections import defaultdict
import math
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import (accuracy_score, confusion_matrix, f1_score,
                             precision_score, recall_score)


In [36]:
# importing from CSV file
data = pd.read_csv("../siren_data_train.csv", sep=",")
# print(f"Data size: {training_data.memory_usage().sum() / 1e6:.2f} MB")
data['heard'] = data['heard'].replace({'hearing':1, 'not_hearing':0})

# Calculate the distance to nearest horn
list_loc_horn = []
list_loc_person = []
list_distance_to_horn = []

x_cor_horn = data["near_x"]
y_cor_horn = data["near_y"]

x_cor_person = data["xcoor"]
y_cor_person = data["ycoor"]

for row in range(len(x_cor_horn)):
    loc_horn = [x_cor_horn[row], y_cor_horn[row]]
    list_loc_horn.append(loc_horn)

    loc_person = [x_cor_person[row], y_cor_person[row]]
    list_loc_person.append(loc_person)

for i in range(len(x_cor_horn)):
    
    coordinate_horn = list_loc_horn[i] 
    coordinate_person = list_loc_person[i]
    distance_to_horn = math.dist(coordinate_horn,coordinate_person)
    list_distance_to_horn.append(distance_to_horn)
   
data["distance to nearest horn"] = list_distance_to_horn
print("The first 10 rows in the data (distance to nearest horn included):")
data.head(10)

data['distance to nearest horn'] = list_distance_to_horn
data = data.drop(['xcoor','ycoor','near_x','near_y'],axis=1)
#print(data)

# Create test set
X = data.drop(['heard'], axis=1)
y = data['heard']
#print(y)

# Splitting into 75% training and 25% test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 42, shuffle=True)


The first 10 rows in the data (distance to nearest horn included):


In [39]:
# K-NN
#Tuning model
params = {'n_neighbors': range(1,50)}
knn = KNeighborsClassifier ()

grid_search = GridSearchCV(knn, params, cv=5, scoring='f1')
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_
print(best_params)
f1_scores = grid_search.cv_results_['mean_test_score']
print('f1:',f1_scores )
params = grid_search.cv_results_['params']
print(params)
best_knn = grid_search.best_estimator_
accuracy = best_knn.score(X_test, y_test)
print('Accuracy: =', accuracy)


2091    1
150     1
756     1
1650    0
3960    1
       ..
3772    1
5191    1
5226    1
5390    1
860     1
Name: heard, Length: 4282, dtype: int64


/Users/sofiapettersson/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/sofiapettersson/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value N

{'n_neighbors': 40}
f1: [0.90207734 0.86888833 0.92512018 0.92055425 0.92960228 0.92638583
 0.9327784  0.93140434 0.93452357 0.93387289 0.93414926 0.93410646
 0.93469903 0.9344418  0.93508934 0.93521141 0.93569477 0.93600157
 0.93575271 0.9362343  0.93636211 0.93629006 0.93681295 0.93661923
 0.93698598 0.93710789 0.93714139 0.93728075 0.93702231 0.93700225
 0.93702231 0.93700534 0.93702231 0.93716146 0.93688336 0.93729752
 0.93702231 0.93729752 0.93704173 0.93729829 0.93690388 0.93702306
 0.93662881 0.93660905 0.93666549 0.93691996 0.93682179 0.93650986
 0.93666616]
[{'n_neighbors': 1}, {'n_neighbors': 2}, {'n_neighbors': 3}, {'n_neighbors': 4}, {'n_neighbors': 5}, {'n_neighbors': 6}, {'n_neighbors': 7}, {'n_neighbors': 8}, {'n_neighbors': 9}, {'n_neighbors': 10}, {'n_neighbors': 11}, {'n_neighbors': 12}, {'n_neighbors': 13}, {'n_neighbors': 14}, {'n_neighbors': 15}, {'n_neighbors': 16}, {'n_neighbors': 17}, {'n_neighbors': 18}, {'n_neighbors': 19}, {'n_neighbors': 20}, {'n_neighbors':

/Users/sofiapettersson/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [38]:
# Performance metrics
y_pred = best_knn.predict(X_test)

def print_performance_metrics(y_true, y_pred):
    print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred), "\n")
    print(f"Accuracy: {accuracy_score(y_true, y_pred):.2}")
    print(f"Recall: {recall_score(y_true, y_pred):.2}")
    print(f"Precision: {precision_score(y_true, y_pred):.2}")
    print(f"F1: {f1_score(y_true, y_pred):.2}")

print_performance_metrics(y_test, y_pred)

Confusion Matrix:
 [[ 231  123]
 [  21 1053]] 

Accuracy: 0.9
Recall: 0.98
Precision: 0.9
F1: 0.94


/Users/sofiapettersson/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
